In [1]:
import numpy as np
import pandas as pd
from simulation.udg import UDG, UDGConfiguration, VARIANT_NONUNIFORM
from simulation.sampler import Simulator, ScenarioEvent, Scenario
import itables.interactive

# Scénario de grosse décroissance avant recroissance plus tard.
scenario = Scenario([
        ScenarioEvent.SUPER_DECREASING,  # TG21.
        ScenarioEvent.STABLE, # TG22
        ScenarioEvent.INCREASING, # TG23
        ScenarioEvent.INCREASING, # TG24
        ScenarioEvent.STABLE, # TG25
        ScenarioEvent.STABLE # TG26
        ])

def extract_kpi(raw_simulation_result):
    cats = ["A", "B", "C"]
    kpis = {}
    for c in cats:
        individuals = [i for i in raw_simulation_result if i['cat'] == c]
        c_loge = sum(1 for individual in individuals if individual['thurne'])
        kpis[f"{c}_total"] = len(individuals)
        kpis[f"{c}_logé"] = c_loge

    return kpis

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
N = 1500
means = []
# 7500 ans de simulations, incroyable, n'est ce pas?
for _ in range(N):
    # Tel que décrit par Milton.
    simulator = Simulator(
            UDG(UDGConfiguration(identifier=VARIANT_NONUNIFORM, function=lambda x: 100*x)),
            scenario)
    raw_results = simulator.run(print_evaluation=False)
    kpis = []
    for rr in raw_results:
        kpis.append(extract_kpi(rr["result"]))
    # pd.DataFrame(kpis, index=range(5)).plot(kind='bar', figsize=(10, 10))

    classements = [pd.DataFrame(
        [(idx + 1, y['person'].id, y['probability'], y['thurne']) for idx, y in enumerate(x['result'])],
        columns=['Classement', 'Identifiant', 'Probabilité', 'Thurné']) for x in raw_results]

    over_years = pd.concat([classement[['Identifiant', 'Probabilité', 'Thurné']] for classement in classements])
    means.append(over_years.groupby("Identifiant").sum()["Thurné"].mean())

print(means)

[1.4394904458598725, 1.1815286624203822, 1.015923566878981, 1.1035031847133758, 1.515923566878981, 1.393312101910828, 1.3439490445859872, 1.0907643312101911, 1.0127388535031847, 1.3694267515923566, 1.5955414012738853, 1.1098726114649682, 1.0875796178343948, 1.1528662420382165, 1.6289808917197452, 1.3073248407643312, 1.2754777070063694, 1.2054140127388535, 1.589171974522293, 1.6289808917197452, 1.2738853503184713, 1.2882165605095541, 1.2404458598726114, 1.5414012738853504, 1.6512738853503184, 1.0764331210191083, 1.4920382165605095, 1.5923566878980893, 1.4044585987261147, 1.6210191082802548, 1.035031847133758, 1.4585987261146496, 1.6576433121019107, 1.0398089171974523, 1.6019108280254777, 0.9554140127388535, 1.0509554140127388, 1.1289808917197452, 1.6035031847133758, 1.4156050955414012, 1.6687898089171975, 1.0079617834394905, 1.0541401273885351, 1.6146496815286624, 1.694267515923567, 1.3264331210191083, 1.6019108280254777, 1.4060509554140128, 1.0207006369426752, 1.5429936305732483, 1.165

In [10]:
pd.DataFrame(means).describe()

,0


In [ ]:
help(pd.DataFrame(kpis, index=range(5)).plot)

In [ ]:
pd.DataFrame(kpis, index=range(5)).plot